In [1]:
import asyncio
import json
import os
from io import StringIO

import openai
import pandas as pd
import requests
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.chat_models import AzureChatOpenAI, ChatOpenAI
from langchain.chains import LLMChain, SequentialChain
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.llms import OpenAI
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
from crewai import Agent, Crew, Process, Task
from textwrap import dedent
from tqdm import tqdm

In [2]:
with open('../utils/openai.json') as jsonfile:
    credentials = json.load(jsonfile)

In [ ]:
model = AzureChatOpenAI(azure_endpoint=credentials['OpenAI']['base'],
                      deployment_name=credentials['OpenAI']['deployment_name'],
                      api_key=credentials['OpenAI']['key'],
                      openai_api_type=credentials['OpenAI']['type'],
                      api_version=credentials['OpenAI']['version'])

In [11]:
clientes = {
    'id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
    'nome': ['João', 'Maria', 'Pedro', 'Ana', 'José', 'Carla', 'Fernando', 'Julia', 'Rafael', 'Lúcia', 'Paulo', 'Mariana', 'Diego', 'Camila', 'Lucas'],
    'idade': [32, 45, 28, 35, 40, 27, 39, 31, 48, 26, 33, 37, 29, 42, 34],
    'sexo': ['M', 'F', 'M', 'F', 'M', 'F', 'M', 'F', 'M', 'F', 'M', 'F', 'M', 'F', 'M'],
    'estado_civil': ['Solteiro', 'Casado', 'Solteiro', 'Divorciado', 'Casado', 'Solteiro', 'Casado', 'Solteiro', 'Divorciado', 'Casado', 'Solteiro', 'Casado', 'Solteiro', 'Casado', 'Solteiro'],
    'cidade': ['São Paulo', 'Rio de Janeiro', 'Belo Horizonte', 'Brasília', 'Salvador', 'Curitiba', 'Fortaleza', 'Recife', 'Porto Alegre', 'Manaus', 'Goiânia', 'Belém', 'Campinas', 'São Luís', 'Natal'],
    'renda_mensal': [5000, 7000, 4000, 6000, 8000, 4500, 7500, 5500, 9000, 4200, 6800, 5300, 4700, 7200, 5800],
    'gasto_medio_em_alimentacao': [1000, 1500, 800, 1200, 2000, 900, 1700, 1100, 2500, 850, 1400, 1000, 950, 1600, 1300],
    'nivel_de_educacao': ['Ensino Médio', 'Graduação', 'Pós-Graduação', 'Pós-Graduação', 'Ensino Médio', 'Graduação', 'Graduação', 'Ensino Médio', 'Pós-Graduação', 'Graduação', 'Ensino Médio', 'Graduação', 'Pós-Graduação', 'Graduação', 'Ensino Médio'],
    'interesse_em_produtos_naturais': ['Sim', 'Sim', 'Não', 'Sim', 'Sim', 'Não', 'Não', 'Sim', 'Sim', 'Não', 'Sim', 'Não', 'Sim', 'Não', 'Sim'],
    'frequencia_de_compras_por_mes': [10, 8, 12, 6, 15, 7, 11, 9, 14, 5, 13, 8, 7, 12, 9],
    'numero_de_filhos': [0, 2, 1, 3, 2, 0, 2, 1, 2, 0, 1, 2, 0, 3, 1],
    'tipo_de_moradia': ['Casa', 'Apartamento', 'Apartamento', 'Casa', 'Casa', 'Apartamento', 'Casa', 'Apartamento', 'Casa', 'Apartamento', 'Casa', 'Apartamento', 'Casa', 'Apartamento', 'Casa'],
    'possui_animal_de_estimacao': ['Sim', 'Não', 'Sim', 'Sim', 'Não', 'Sim', 'Não', 'Sim', 'Sim', 'Não', 'Sim', 'Sim', 'Não', 'Sim', 'Não'],
    'grau_de_satisfacao_com_alimentacao': [8, 9, 7, 8, 9, 6, 8, 9, 7, 8, 9, 7, 8, 9, 7],
    'preferencia_de_comida': ['Italiana', 'Japonesa', 'Brasileira', 'Vegetariana', 'Churrasco', 'Fast food', 'Pizza', 'Saudável', 'Frutos do Mar', 'Árabe', 'Indiana', 'Sobremesas', 'Mediterrânea', 'Mexicana', 'Regional'],
    'possui_automovel': ['Sim', 'Sim', 'Não', 'Sim', 'Sim', 'Não', 'Sim', 'Sim', 'Sim', 'Não', 'Sim', 'Sim', 'Não', 'Sim', 'Não'],
    'formacao': ['Técnico', 'Graduação', 'Ensino Médio', 'Pós-Graduação', 'Graduação', 'Ensino Médio', 'Graduação', 'Pós-Graduação', 'Graduação', 'Ensino Médio', 'Graduação', 'Pós-Graduação', 'Graduação', 'Ensino Médio', 'Pós-Graduação']
}

In [13]:
df = pd.DataFrame(clientes)
df.head(5)

,id,nome,idade,sexo,estado_civil,cidade,renda_mensal,gasto_medio_em_alimentacao,nivel_de_educacao,interesse_em_produtos_naturais,frequencia_de_compras_por_mes,numero_de_filhos,tipo_de_moradia,possui_animal_de_estimacao,grau_de_satisfacao_com_alimentacao,preferencia_de_comida,possui_automovel,formacao
0,1,João,32,M,Solteiro,São Paulo,5000,1000,Ensino Médio,Sim,10,0,Casa,Sim,8,Italiana,Sim,Técnico
1,2,Maria,45,F,Casado,Rio de Janeiro,7000,1500,Graduação,Sim,8,2,Apartamento,Não,9,Japonesa,Sim,Graduação
2,3,Pedro,28,M,Solteiro,Belo Horizonte,4000,800,Pós-Graduação,Não,12,1,Apartamento,Sim,7,Brasileira,Não,Ensino Médio
3,4,Ana,35,F,Divorciado,Brasília,6000,1200,Pós-Graduação,Sim,6,3,Casa,Sim,8,Vegetariana,Sim,Pós-Graduação
4,5,José,40,M,Casado,Salvador,8000,2000,Ensino Médio,Sim,15,2,Casa,Não,9,Churrasco,Sim,Graduação


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 18 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   id                                  15 non-null     int64 
 1   nome                                15 non-null     object
 2   idade                               15 non-null     int64 
 3   sexo                                15 non-null     object
 4   estado_civil                        15 non-null     object
 5   cidade                              15 non-null     object
 6   renda_mensal                        15 non-null     int64 
 7   gasto_medio_em_alimentacao          15 non-null     int64 
 8   nivel_de_educacao                   15 non-null     object
 9   interesse_em_produtos_naturais      15 non-null     object
 10  frequencia_de_compras_por_mes       15 non-null     int64 
 11  numero_de_filhos                    15 non-null     int64 
 

In [ ]:
data_profiler_agent = Agent(
    role='Data Profiler',
    goal='''From limited data, logically deduce conclusions about customers' characteristics and preferences.''',
    backstory='''You are an experienced data analyst specialized in behavioral psychology, with decades of experience in inferring consumer profiles based on their demographic information, consumption habits, and personal preferences.''',
    llm=model,
    verbose=True,
    allow_delegation=True
)

product_matching_agent = Agent(
    role='Product Matching Specialist',
    goal='''Match each product to the customer's needs and preferences, using data to identify the perfect fit between product and consumer.''',
    backstory='''You have exceptional knowledge of products and deeply understand how different characteristics of consumers (such as income, number of children, type of housing) affect their appreciation of products.''',
    llm=model,
    verbose=True,
    allow_delegation=True
)

promotional_director_agent = Agent(
    role="Promotional Director",
    goal=dedent("""\
        Oversee the work done by your team to ensure that the advertising campaigns are as effective as possible and aligned with the product's goals, using data to review, approve, and, if necessary, delegate additional tasks to make decisions."""),
    backstory=dedent("""\
        You are the Promotional Director of a large retailer, launching a personalized advertising campaign. Your role is to ensure that your team is creating the most relevant content for customers, based on detailed analyses of their consumption profiles and preferences."""),
    llm=model,
    verbose=True
)

creative_ad_creator_agent = Agent(
    role="Creative Ad Creator",
    goal=dedent("""\
        Develop innovative and engaging advertising content, focusing on creating customer-specific ad copies based on a detailed analysis of consumer data."""),
    backstory=dedent("""\
        As a Creative Ad Creator at a top-tier digital marketing agency, you excel in crafting advertisements that resonate with potential customers. Your expertise lies in turning marketing strategies into engaging stories that capture attention and inspire buying actions, using demographic and behavioral data to personalize each message."""),
    llm=model,
    verbose=True
)

In [ ]:
from textwrap import dedent

def get_profiling_task(agent, customer_description):
    return Task(description=f"""\
        You're tasked with creating a targeted marketing campaign tailored to our customers' specific characteristics. 
        Based on the given customer description: {customer_description}, build an understanding of our customer using available demographic, behavioral, and transactional data. 
        Analyze these data points to deduce their psychographic profile, considering their personalities, values, interests, lifestyle, and needs.
        """,
        agent=agent,
        expected_output="""
        Your final answer MUST be exactly a short description of the customer's psychographic profile. Format it like this example:
        <Psychographic> : <Psychographic description>
        """
    )

def get_ad_campaign_task(agent, customer_profile, products):
    return Task(description=dedent(f"""\
        Develop a targeted marketing campaign based on the customer profiles we have.
        For each customer, select exactly three products that are most likely to appeal to them, based on the customer profile: {customer_profile}.
        Review these selections thoroughly, ensure alignment with customer needs, and delegate further analysis if necessary.
        List all products participating in the campaign: {products}.
        """),
        agent=agent,
        expected_output="""Your final answer MUST be exactly 3 products from the list, each justified with a brief explanation of why it is suitable for the customer:
        <Product 1> : <Reason>
        <Product 2> : <Reason>
        <Product 3> : <Reason>
        """
    )

def get_ad_campaign_written_task(agent, selection):
    return Task(description=dedent(f"""\
        Create engaging promotional content for a targeted marketing campaign. 
        Based on the selected three products tailored specifically to our customer: {selection}, craft promotional messages that will resonate with the customer and inspire them to make a purchase.
        Develop content that is succinct enough to fit into three paragraphs, making it suitable for integration into a comprehensive advertising letter.
        """),
        agent=agent,
        expected_output="""Your final answer MUST include 3 promotional messages for the selected products, formatted as follows:
        <Product 1> : <Promotional message>
        <Product 2> : <Promotional message>
        <Product 3> : <Promotional message>
        """
    )

# Assign tasks to specific agents
profiling_task = get_profiling_task(data_profiler_agent, "Here describe the customer using your data frame")
ad_campaign_task = get_ad_campaign_task(promotional_director_agent, "Here is the customer's profile", "List of products")
ad_campaign_written_task = get_ad_campaign_written_task(creative_ad_creator_agent, "Selected products and reasons")

In [ ]:
df_output_list = []  # to store results

for index, row in df_customers.iterrows():
    print('############################################## ' + str(index))
    customer_description = f'''The customer was born in {row['birth_year']}.
    Its sex is {row['sex']}.
    Its marital status is {row['marital_status']}.
    Its yearly household income percentile is {row['yearly_household_income_percentile']}.
    The customer has {row['number_of_toddlers']} toddlers.
    The customer has {row['number_of_teens']} teenage kids.
    The customer highest education is {row['highest_education']}.
    The customer spends on average {row['monthly_spend_on_wine']} $ on wine.
    The customer spends on average {row['monthly_spend_on_vegetables']} $ on vegetables.
    The customer spends on average {row['monthly_spend_on_toys']} $ on toys.
    The customer's coupon use is {row['last_month_coupon_use']}.
    '''

    # Adapted tasks using new agent names and functions
    profiling_task = get_profiling_task(data_profiler_agent, customer_description)
    profiling_crew = Crew(
        agents=[data_profiler_agent, promotional_director_agent],
        tasks=[profiling_task],
        verbose=1,
        process=Process.sequential
    )
    profiling_result = profiling_crew.kickoff()

    ad_campaign_task = get_ad_campaign_task(promotional_director_agent, profiling_result, "List of products")
    targeting_crew = Crew(
        agents=[product_matching_agent, promotional_director_agent],
        tasks=[ad_campaign_task],
        verbose=1,
        process=Process.sequential
    )
    targeting_result = targeting_crew.kickoff()

    ad_campaign_written_task = get_ad_campaign_written_task(creative_ad_creator_agent, targeting_result)
    copywriting_crew = Crew(
        agents=[creative_ad_creator_agent, promotional_director_agent],
        tasks=[ad_campaign_written_task],
        verbose=1,
        process=Process.sequential
    )
    copywriting_result = copywriting_crew.kickoff()

    # create one line in output df
    df_output_list.append({
        'customer': customer_description,
        'profile': profiling_result,
        'targeted_products': targeting_result,
        'promo_msg': copywriting_result,
    })

# loop ended, collect results in dataframe
df_output = pd.DataFrame(df_output_list)